In [5]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint

print( tf.VERSION )


1.13.1


In [2]:
encoder_input_data = np.load( 'encoder_input_data_friends.npy' )
decoder_input_data = np.load( 'decoder_input_data_friends.npy' )
decoder_target_data = np.load( 'decoder_target_data_friends.npy' )

embedding_matrix = np.load('embedding_matrix_friends.npy' ) 

tokenizer = pickle.load( open('tokenizer_friends.pkl' , 'rb'))

num_tokens = len( tokenizer.word_index )+1
word_dict = tokenizer.word_index

max_question_len = encoder_input_data.shape[1]
max_answer_len = decoder_input_data.shape[1]

print( 'Max length of question is {}'.format( max_question_len) )
print( 'Max length of answer is {}'.format( max_answer_len) )

print( encoder_input_data.shape )
print( decoder_input_data.shape )
print( decoder_target_data.shape )

Max length of question is 16
Max length of answer is 19
(24380, 16)
(24380, 19)
(24380, 19, 8871)


In [3]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_tokens, 200 , mask_zero=True , weights=[embedding_matrix] ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_tokens, 200 , mask_zero=True, weights=[embedding_matrix]) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    1774200     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    1774200     input_2[0][0]                    
_____________________________________

In [6]:
filepath="weights-friends-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [11]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=50, epochs=100, callbacks=callbacks_list ) 
model.save( 'model_friends_epochs100.h5' ) 


Epoch 1/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.2528
Epoch 00001: loss improved from 1.25536 to 1.25281, saving model to weights-friends-01-1.2528.hdf5
24380/24380 [==============================] - 73s 3ms/sample - loss: 1.2528
Epoch 2/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.2464
Epoch 00002: loss improved from 1.25281 to 1.24685, saving model to weights-friends-02-1.2469.hdf5
24380/24380 [==============================] - 89s 4ms/sample - loss: 1.2469
Epoch 3/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.2415
Epoch 00003: loss improved from 1.24685 to 1.24151, saving model to weights-friends-03-1.2415.hdf5
24380/24380 [==============================] - 95s 4ms/sample - loss: 1.2415
Epoch 4/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.2349
Epoch 00004: loss improved from 1.24151 to 1.23529, saving model to weights-friends-04-1.2353.hdf5
24380/24380 [==============================] 

24380/24380 [==============================] - 70s 3ms/sample - loss: 1.1429
Epoch 33/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.1422
Epoch 00033: loss improved from 1.14292 to 1.14156, saving model to weights-friends-33-1.1416.hdf5
24380/24380 [==============================] - 73s 3ms/sample - loss: 1.1416
Epoch 34/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.1390
Epoch 00034: loss improved from 1.14156 to 1.13883, saving model to weights-friends-34-1.1388.hdf5
24380/24380 [==============================] - 89s 4ms/sample - loss: 1.1388
Epoch 35/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.1327
Epoch 00035: loss improved from 1.13883 to 1.13330, saving model to weights-friends-35-1.1333.hdf5
24380/24380 [==============================] - 69s 3ms/sample - loss: 1.1333
Epoch 36/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.1334
Epoch 00036: loss did not improve from 1.13330
24380/24380 [===

Epoch 65/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.0814
Epoch 00065: loss improved from 1.08259 to 1.08113, saving model to weights-friends-65-1.0811.hdf5
24380/24380 [==============================] - 70s 3ms/sample - loss: 1.0811
Epoch 66/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.0791
Epoch 00066: loss improved from 1.08113 to 1.07854, saving model to weights-friends-66-1.0785.hdf5
24380/24380 [==============================] - 76s 3ms/sample - loss: 1.0785
Epoch 67/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.0768
Epoch 00067: loss improved from 1.07854 to 1.07702, saving model to weights-friends-67-1.0770.hdf5
24380/24380 [==============================] - 69s 3ms/sample - loss: 1.0770
Epoch 68/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.0751
Epoch 00068: loss improved from 1.07702 to 1.07505, saving model to weights-friends-68-1.0750.hdf5
24380/24380 [============================

Epoch 98/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.0393
Epoch 00098: loss did not improve from 1.03863
24380/24380 [==============================] - 70s 3ms/sample - loss: 1.0395
Epoch 99/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.0368
Epoch 00099: loss improved from 1.03863 to 1.03672, saving model to weights-friends-99-1.0367.hdf5
24380/24380 [==============================] - 71s 3ms/sample - loss: 1.0367
Epoch 100/100
24350/24380 [============================>.] - ETA: 0s - loss: 1.0348
Epoch 00100: loss improved from 1.03672 to 1.03482, saving model to weights-friends-100-1.0348.hdf5
24380/24380 [==============================] - 74s 3ms/sample - loss: 1.0348


In [12]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [13]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        try:
            tokens_list.append( word_dict[ word ] ) 
        except:
            continue
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_question_len , padding='post')

In [14]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_answer_len:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : hi!
 i know i have to just here end
Enter question : hi
 hi end
Enter question :  
 i know i have to just here end
Enter question : what's up
 this uh go on the that's a big deal end
Enter question : up
 you will hey end
Enter question : whats up
 you will hey end
Enter question : what up
 like i got get a end
Enter question : where have you been
 oh just had a baby end
Enter question : you just had a baby ?
 honey i got a end
Enter question : what you got
 wanna have a come on him end
